In [ ]:
# Extract data from web(weather) using web scrapping

import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

# Define the URL
URL = "https://www.timeanddate.com/weather/usa/new-york/historic"

# Send a GET request to fetch the HTML content
response = requests.get(URL)

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# Find all table rows (<tr> elements)
rows = soup.find_all('tr')

# Find the table containing the weather data (adjust selector based on actual table structure)
table = soup.find('table')  # You might need to add specific classes or IDs to locate the correct table

# Check if the table is found
if table:
    # Extract table headers (column names)
    headers = [header.text.strip() for header in table.find_all('th')]

    # Print the headers for debugging
    print("Extracted Headers:", headers)
else:
    print("Table not found on the webpage.")


# List to store scraped data
web_scraped_data = []

# Loop through 
for row in rows[1:]:  # Skip the header row
    columns = row.find_all('td')  # Find all table data cells
    
    if len(columns) >= 7:  # Ensure there are enough columns
        try:
            # Extract and clean the data for each column
            temp = columns[1].text.strip()  
            weather = columns[2].text.strip()
            wind = columns[3].text.strip()
            humidity = columns[5].text.strip()
            barometer = columns[6].text.strip()
            visibility = columns[7].text.strip()

            # Append the data as a dictionary to the list
            web_scraped_data.append({
                'Temp': temp,
                'Weather': weather,
                'Wind': wind,
                'Humidity': humidity,
                'Barometer': barometer,
                'Visibility': visibility,
            })
        except Exception as e:
            print(f"Error processing row: {[col.text.strip() for col in columns]}, Error: {e}")

# Convert the list of dictionaries into a Pandas DataFrame
if web_scraped_data:
    web_scraping_df = pd.DataFrame(web_scraped_data)
    print(web_scraping_df)
else:
    print("No valid data was scraped.")




In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Define the URL
URL = "https://www.timeanddate.com/weather/usa/new-york/historic"

# Send a GET request to fetch the HTML content
response = requests.get(URL)

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# Find the table containing the weather data (adjust selector based on actual table structure)
table = soup.find('table')  # You might need to add specific classes or IDs to locate the correct table

# Check if the table is found
if table:
    # Extract table headers (column names)
    headers = [header.text.strip() for header in table.find_all('th')]

    # Print the headers for debugging
    print("Extracted Headers:", headers)
else:
    print("Table not found on the webpage.")

# List to store scraped data
web_scraped_data = []

# Loop through 
for row in rows[1:]:  # Skip the header row
    columns = row.find_all('td')  # Find all table data cells
    
    if len(columns) >= 7:  # Ensure there are enough columns
        try:
            # Extract and clean the data for each column
            time = columns[0].text.strip()
            temp = columns[1].text.strip()  
            weather = columns[2].text.strip()
            wind = columns[3].text.strip()
            humidity = columns[4].text.strip()
            barometer = columns[5].text.strip()
            visibility = columns[6].text.strip()

            # Append the data as a dictionary to the list
            web_scraped_data.append({
                'Time': time,
                'Temp': temp,
                'Weather': weather,
                'Wind': wind,
                'Humidity': humidity,
                'Barometer': barometer,
                'Visibility': visibility,
            })
        except Exception as e:
            print(f"Error processing row: {[col.text.strip() for col in columns]}, Error: {e}")

# Convert the list of dictionaries into a Pandas DataFrame
if web_scraped_data:
    web_scraping_df = pd.DataFrame(web_scraped_data)
    print(web_scraping_df)
else:
    print("No valid data was scraped.")





Extracted Headers: ['', 'Temperature', 'Humidity', 'Pressure', 'High', 'Low', 'Average']
   Time  Temp                     Weather     Wind Humidity Barometer  \
0        6 °C                      Clear.      N/A        ↑       65%   
1        6 °C                   Overcast.   9 km/h        ↑       65%   
2        6 °C                      Clear.  22 km/h        ↑       68%   
3        6 °C             Passing clouds.  19 km/h        ↑       74%   
4        6 °C             Passing clouds.  22 km/h        ↑       74%   
5        6 °C             Passing clouds.      N/A        ↑       76%   
6        6 °C             Passing clouds.  20 km/h        ↑       76%   
7        6 °C              Partly cloudy.  19 km/h        ↑       76%   
8        6 °C                   Overcast.  20 km/h        ↑       82%   
9        7 °C                   Overcast.  20 km/h        ↑       82%   
10       7 °C       More clouds than sun.  13 km/h        ↑       80%   
11       7 °C                 Low c